# **Rede Convolucional**

Implementação de CNN para a identificação da idade e género tendo como base imagens das faces provenientes do IMDB e da Wikipedia.

# **Carregamento dos Dados**

In [4]:
import pandas as pd
from tensorflow.keras import layers, models

In [5]:
features = pd.read_csv('./result.csv', header=None)
colunas = ['foto', 'idade', 'genero']
features.columns = colunas

In [6]:
features

,foto,idade,genero
0,17/10000217_1981-05-05_2009.jpg,28,1.0
1,48/10000548_1925-04-04_1964.jpg,39,1.0
2,12/100012_1948-07-03_2008.jpg,60,1.0
3,65/10001965_1930-05-23_1961.jpg,31,1.0
4,16/10002116_1971-05-31_2012.jpg,41,0.0
...,...,...,...
62323,49/9996949_1937-04-17_1963.jpg,26,1.0
62324,32/9997032_1947-07-30_1970.jpg,23,1.0
62325,09/9998109_1972-12-27_2013.jpg,41,1.0
62326,00/9999400_1981-12-13_2011.jpg,30,1.0


In [7]:
features = features.set_index('foto')
features.head()

,idade,genero
foto,,
17/10000217_1981-05-05_2009.jpg,28,1.0
48/10000548_1925-04-04_1964.jpg,39,1.0
12/100012_1948-07-03_2008.jpg,60,1.0
65/10001965_1930-05-23_1961.jpg,31,1.0
16/10002116_1971-05-31_2012.jpg,41,0.0


Remoção de entradas com valores sem sentido para a idade.

In [8]:
(features.idade < 0).sum()

1198

In [9]:
print(features.shape)
features = features.dropna()
print(features.shape)

(62328, 2)
(59685, 2)


In [10]:
features = features[features.idade > 0] 
print(features.shape)
features = features[features.idade < 100]
print(features.shape)

(58460, 2)
(57932, 2)


# Modelo

A rede convolucional é composta por uma alternância de layers Conv2D, com ativação <i>relu</i>, e MaxPooling, sendo os inputs iniciais são de tamanho 150x150, com 3 canais, dado tratarem-se de imagens RGB. Temos por fim uma layer Flatten e duas camadas Dense.
De notar, que a camada de output é constituída por um único neurónio com ativação linear, visto tratar-se de uma regressão.


In [14]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='linear'))

Observação das alterações à dimensão dos <i>feature maps</i> em cada camada:

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)      

Na compilação do modelo usámos o optimizer <b>RMSprop</b> com uma taxa de aprendizagem de 0.0001. Tratando-se de um problema de regressão é aplicada a <i>loss function</i> mse(erro quadrático médio) e a métrica mae(erro absoluto médio).

In [20]:
from tensorflow.keras import optimizers

model.compile(loss='mse',optimizer=optimizers.RMSprop(lr=1e-4),metrics=['mae'])